## Dependencies

In [ ]:
import os
import shutil
import cv2
import random
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from random import randint

## Data Load

In [6]:
keep_stanford40 = ["applauding", "climbing", "drinking", "jumping", "pouring_liquid", "riding_a_bike", "riding_a_horse", 
        "running", "shooting_an_arrow", "smoking", "throwing_frisby", "waving_hands"]

with open('Stanford40/ImageSplits/train.txt', 'r') as f:
    # We won't use these splits but split them ourselves
    train_files = [file_name for file_name in list(map(str.strip, f.readlines())) if '_'.join(file_name.split('_')[:-1]) in keep_stanford40]
    train_labels = ['_'.join(name.split('_')[:-1]) for name in train_files]

    
with open('Stanford40/ImageSplits/test.txt', 'r') as f:
    # We won't use these splits but split them ourselves
    test_files = [file_name for file_name in list(map(str.strip, f.readlines())) if '_'.join(file_name.split('_')[:-1]) in keep_stanford40]
    test_labels = ['_'.join(name.split('_')[:-1]) for name in test_files]

# Combine the splits and split for keeping more images in the training set than the test set.
all_files = train_files + test_files
all_labels = train_labels + test_labels
train_files, test_files = train_test_split(all_files, test_size=0.1, random_state=0, stratify=all_labels)
train_labels = ['_'.join(name.split('_')[:-1]) for name in train_files]
test_labels = ['_'.join(name.split('_')[:-1]) for name in test_files]
action_categories = sorted(list(set(train_labels)))

In [7]:
train_files, val_files = train_test_split(train_files, test_size=0.1, random_state=0, stratify=train_labels)
train_labels = ['_'.join(name.split('_')[:-1]) for name in train_files]
val_labels = ['_'.join(name.split('_')[:-1]) for name in val_files]

In [8]:
str_to_int = {"applauding": 0, "climbing": 1, "drinking": 2, "jumping":3 , "pouring_liquid":4, "riding_a_bike":5, "riding_a_horse":6, 
              "running":7, "shooting_an_arrow":8, "smoking":9, "throwing_frisby":10, "waving_hands":11}

y_train = np.array([str_to_int[label] for label in train_labels])
y_val = np.array([str_to_int[label] for label in val_labels])
y_test = np.array([str_to_int[label] for label in test_labels])

In [ ]:
x_train = []
for i, file in enumerate(train_files):
    img = cv2.imread(os.path.join('Stanford40/JPEGImages/', file))
    img = cv2.resize(img, (112, 112))
    x_train.append(img)
x_train = np.asarray(x_train)/255.

In [ ]:
x_val = []
for i, file in enumerate(val_files):
    img = cv2.imread(os.path.join('Stanford40/JPEGImages/', file))
    img = cv2.resize(img, (112, 112))
    x_val.append(img)
x_val = np.asarray(x_val)/255.

In [38]:
x_test = []
for i, file in enumerate(test_files):
    img = cv2.imread(os.path.join('Stanford40/JPEGImages/', file))
    img = cv2.resize(img, (112, 112))
    x_test.append(img)
x_test = np.asarray(x_test)/255.

In [ ]:
# Fetching all the files to directory
for file_name in train_files:
   shutil.copy("Stanford40/JPEGImages/"+file_name, "Stanford40/data/train/"+file_name)

# Fetching all the files to directory
for file_name in val_files:
   shutil.copy("Stanford40/JPEGImages/"+file_name, "Stanford40/data/val/"+file_name)

# Fetching all the files to directory
for file_name in test_files:
   shutil.copy("Stanford40/JPEGImages/"+file_name, "Stanford40/data/test/"+file_name)

## Data Augmentation

In [ ]:
# Define the types of augmentations to perform
augmentations = {
    'rotate': (-30, 30),  # rotation range in degrees
    'scale': (0.5, 1.5),  # scaling range
    'shear': (-20, 20),   # shearing range in degrees
    'brightness': (0.5, 1.5),  # brightness range
    'flip': True         # horizontal flipping
}

# Define the number of augmented images to create per original image
num_augmentations = 3

# Define a function to perform data augmentation
def augment_image(img_path, augmentations, num_augmentations):
    # Load the original image using OpenCV
    img = cv2.imread(img_path)

    # Loop over the number of augmentations to create
    for i in range(num_augmentations):
        # Copy the original image to apply the augmentations to
        augmented_img = img.copy()

        # Apply the specified augmentations
        if augmentations['rotate']:
            angle = randint(*augmentations['rotate'])
            rows, cols, _ = img.shape
            M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
            augmented_img = cv2.warpAffine(augmented_img, M, (cols, rows))

        if augmentations['scale']:
            scale = randint(*augmentations['scale'])
            rows, cols, _ = img.shape
            M = cv2.getAffineTransform(
                np.float32([[0, 0], [cols-1, 0], [0, rows-1]]),
                np.float32([[0, 0], [cols*scale-1, 0], [0, rows*scale-1]])
            )
            augmented_img = cv2.warpAffine(augmented_img, M, (cols, rows))

        if augmentations['shear']:
            angle = randint(*augmentations['shear'])
            rows, cols, _ = img.shape
            M = np.float32([[1, np.tan(angle*np.pi/180), 0], [0, 1, 0]])
            augmented_img = cv2.warpAffine(augmented_img, M, (cols, rows))

        if augmentations['brightness']:
            brightness = randint(*augmentations['brightness'])
            hsv = cv2.cvtColor(augmented_img, cv2.COLOR_BGR2HSV)
            hsv[:, :, 2] = np.clip(hsv[:, :, 2]*brightness, 0, 255)
            augmented_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        if augmentations['flip']:
            if randint(0, 1) == 1:
                augmented_img = cv2.flip(augmented_img, 1)

        # Save the augmented image to a file
        file_name, ext = os.path.splitext(img_path)
        save_path = f"{file_name}_augmented_{i+1}{ext}"
        cv2.imwrite(save_path, augmented_img)

In [43]:
def augment_img(img, method):
    img_copy = img.copy()
    
    if method == 'rotate':
        angle = randint(*(-30, 30))
        rows, cols, _ = img.shape
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        augmented_img = cv2.warpAffine(img_copy, M, (cols, rows))

    if method == 'scale':
        scale = random.choice([0.5, 1.5, 2, 2.5])
        rows, cols, _ = img.shape
        M = cv2.getAffineTransform(
            np.float32([[0, 0], [cols-1, 0], [0, rows-1]]),
            np.float32([[0, 0], [cols*scale-1, 0], [0, rows*scale-1]])
        )
        augmented_img = cv2.warpAffine(img_copy, M, (cols, rows))

    if method == 'shear':
        angle = randint(*(-20, 20))
        rows, cols, _ = img.shape
        M = np.float32([[1, np.tan(angle*np.pi/180), 0], [0, 1, 0]])
        augmented_img = cv2.warpAffine(img_copy, M, (cols, rows))
        
    return augmented_img

In [61]:
x_test = []
for i, file in enumerate(test_files):
    img = cv2.imread(os.path.join('Stanford40/JPEGImages/', file))
    img = cv2.resize(img, (112, 112))
    x_test.append(img)

In [62]:
str_to_int = {"applauding": 0, "climbing": 1, "drinking": 2, "jumping":3 , "pouring_liquid":4, "riding_a_bike":5, "riding_a_horse":6, 
              "running":7, "shooting_an_arrow":8, "smoking":9, "throwing_frisby":10, "waving_hands":11}

y_test = [str_to_int[label] for label in test_labels]

In [64]:
methods = ['rotate', 'scale', 'shear']

for i, img in enumerate(x_test[:50]):
    for method in methods:
        augmented_img = augment_img(img, method)
        x_test.append(augmented_img)
        y_test.append(y_test[i])

In [65]:
print(len(x_test))
print(len(y_test))

454
454
